**Title**: Update MoCo Acquisition Label on 
**Date**:  04-03-2020  
**Description**:  
* This notebook is used to rename MoCo Series based on the Scan Name, end with `_moco` prefix 
* Functions: 

1.   `get_scan_moco_label` : Identify the original scan label
2.   `modify_acq_name` : Modify Acquisition name based on the session timepoint
3.  `update_acq_label` : Update the acquisition label on the FW instances





# Install and import dependencies

In [0]:
# Install specific packages required for this notebook
!pip install flywheel-sdk

In [0]:
# Import packages
from getpass import getpass
import logging
import os
import pandas as pd
import flywheel

In [0]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

# Flywheel API Key and Client

Get a API_KEY. More on this at in the Flywheel SDK doc [here](https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/getting_started.html#api-key).

In [0]:
API_KEY = getpass('Enter API_KEY here: ')

Instantiate the Flywheel API client

In [0]:
fw = flywheel.Client(API_KEY or os.environ.get('FW_KEY'))

Show Flywheel logging information

In [0]:
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

# Suggestions on how to structure your notebook

## Constants

In [0]:
project_id = '5e87a92c5201f10c5b44d8aa'

project = fw.get_project(project_id)

# Name for each session and prefix (that has only one run) on the project
session_name_and_acq_prefix = {'DarbyHatch': ['Rest', 'Cry', 'WhyHow'], 'Darbypostpartum':['Rest']}


## Helper functions

In [0]:
def get_scan_moco_label(acq_list, session_name):

  """Return the Acquisition List with the updated label

  Args:
       acq_list : List of dictionaries that consists of 
                  each acquisition label (`acq_label`), 
                  acquisition id (`acq_id`) ,and 
                  series number (`series_number`) 
                  on one Session
       session_name (str):  Session Label of the project
       
    Returns:
       (list): `acq_list` with updated label name if label name is modified, NaN otherwise

  """

  for index, acq in enumerate(acq_list):

    acq_label = acq['acq_label']

    if acq_label.startswith('MoCo') or acq_label.endswith('_moco'):
        scan_acq_label = acq_list[index - 1]['acq_label']
        scan_acq_id = acq_list[index - 1]['acq_id']
        new_scan_label = modify_acq_name(scan_acq_label, session_name)
        acq_list[index-1]['updated_label'] = new_scan_label
        update_acq_label(scan_acq_id, new_scan_label)
        moco_label = new_scan_label + '_moco'
        acq_list[index]['updated_label'] = moco_label
        update_acq_label(acq['acq_id'], moco_label)


  return acq_list

def modify_acq_name(acq_name, tp):

  """Get the modified Acquisition Label Name

  Args:
        acq_name (str) : Name for the Acquisition
        tp (str) : Session Name under the acqusition

    
    Returns:
      (str): `acq_name` with updated value if they matches the prefix 


  """
    separator = '_'

    for session, prefix in session_name_and_acq_prefix.items():
      if tp == session:
        for each in prefix:
          if acq_name.startswith(each):
            acq_name = each
          elif acq_name.endswith('_0'):
            split_name = acq_name.replace('0', 'r2').split(separator)
            acq_name = split_name[0] + '_' + split_name[2]
          elif acq_name.endswith('_1'):
            split_name = acq_name.replace('_1', '_r3').split(separator)
            acq_name = split_name[0] + '_' + split_name[2]
          elif acq_name.endswith('fmri'):
            split_name = acq_name.replace('_fmri', '_r1').split(separator)
            acq_name = split_name[0] + '_' + split_name[1]
    return acq_name


def update_acq_label(acq_id, new_label):
  """Update the acquisition label on the project container

  Args:
        acq_id (str) : Acquisition ID
        new_label (str) : Updated Acquisition Label
        
  """
  new_acq_object = flywheel.models.Acquisition(label = new_label)
  # modifying the acquisition label here 
  fw.modify_acquisition(acquisition_id= acq_id, body = new_acq_object)

## Main script

In [0]:
df = pd.DataFrame()

for session in project.sessions.iter():
    session_name = session.label
    all_acq_list = []
    for acquisition in session.acquisitions.iter():
        acquisition = acquisition.reload()

        for file in acquisition.files:
            if file['type'] == 'dicom':
                item = {
                    'series_number': file.info.get('SeriesNumber'),
                    'acq_label': acquisition.label,
                    'acq_id': acquisition.id
                }
                all_acq_list.append(item)
    # sort the list by each series number
    all_acq_list = sorted(all_acq_list, key=lambda k: k['series_number'])
    acq_list = get_scan_moco_label(all_acq_list, session_name)
    df = df.append(acq_list, ignore_index=True)


## Display Updated Acquisition Info


In [0]:
pd.options.display.max_columns = None

display(df.head(50))